In [1]:
import  pickle
import numpy as np
circuit_paths = [ 'random_transpiled_circuit_3400.pkl', 'circuit20_20.pkl', 'alg_transpiled_circuit_13.pkl',]

In [2]:

circuits = []
for circuit_path in circuit_paths:
    with open(circuit_path,'rb') as f:
        circuits += pickle.load(f)
circuits = circuits[1:]

In [3]:
with open('counts.pkl','rb') as f:
    counts = pickle.load(f)

In [5]:
from circuit import qiskit_to_layered_circuits
dataset = []
reals = []
for cir, count in zip(circuits[:3399] ,counts[:3399]):
    if '00000' not in count:
        continue
    ground_truth_fidelity = count['00000'] / 5000
    if ground_truth_fidelity < 0.2:
        # print('bad')
        continue

    reals.append(ground_truth_fidelity)
    circuit_info = qiskit_to_layered_circuits(cir)
    circuit_info['ground_truth_fidelity'] = ground_truth_fidelity
    dataset.append(circuit_info)

In [8]:
train_dataset, test_dataset = dataset[:-400], dataset[-400:]
len(train_dataset), len(test_dataset), len(circuits[3399:])

(2192, 400, 530)

In [9]:
from utils.backend import Backend
coupling_map = [[4, 3], [3, 4], [2, 3], [3, 2], [1, 2], [2, 1], [0, 1], [1, 0]]
topology = {0: [1], 1: [0, 1], 2: [1,3], 3: [2,4], 4: [3]}
backend = Backend(n_qubits=5, topology=topology, neighbor_info=None, coupling_map=coupling_map,
basis_single_gates = ['id', 'rz', 'sx', 'x'], basis_two_gates = ['cx'], divide = False, decoupling = False)

In [ ]:
from upstream import RandomwalkModel
steps = []
for n_steps in range(4):
    upstream_model = RandomwalkModel(n_steps, 20, backend=backend, travel_directions=('parallel', 'former'))
    upstream_model.train(dataset, multi_process=True, remove_redundancy=n_steps > 1)

    from downstream import FidelityModel

    downstream_model = FidelityModel(upstream_model)
    downstream_model.train(train_dataset, epoch_num = 300)

    test_predicts, test_reals = [] , []
    for circuit_info in test_dataset:

        predict = downstream_model.predict_fidelity(circuit_info)
        circuit_info['predict'] = predict

        test_reals.append(circuit_info['ground_truth_fidelity'])
        test_predicts.append(predict)
    print(np.abs(np.array(test_reals)-np.array(test_predicts)).mean())
    steps.append(np.abs(np.array(test_reals)-np.array(test_predicts)).tolist())


In [ ]:
steps = pickle.dump(steps,open('steps.pkl','wb'))